# Montage Angle Assessment
Varies the 3D orientation (angle in a vertical plane) of a single current dipole positioned between two DISC devices for three electrode configurations. The goal is to visualize how electrode selection (different montage choices) affects measured voltage as the dipole rotates.

**Inputs**:
- Leadfield file: `DISC_30mm_p2-5Sm_MacChr.npz` (update path via `folder` variable)
- Voxel size (`scale`), dipole magnitude (`magnitude`), noise level (`noise`)

**Outputs**:
- Plot `4_montage_angle.png` saved to `<dataset>/outputs` showing voltage vs. dipole angle.

**Procedure Summary**:
1. Load leadfield (fields) and determine electrode count.
2. Define dipole parameters and spatial location between devices.
3. Sweep dipole orientation angle from 0° to 180°.
4. Compute voltages for three selected electrodes.
5. Plot voltage curves and a zero reference line.

Update the `folder` path before running to point at your dataset root containing `leadfields/` and `outputs/` directories.

In [ ]:
### Setup: Imports and Leadfield Loading ###
# This cell:
# 1. Imports required libraries.
# 2. Adds the local project root to sys.path so modules can be found.
# 3. Specifies dataset folder and leadfield file paths (EDIT `folder`).
# 4. Loads the leadfield (electric field) data for DISC electrodes.
# 5. Defines basic acquisition / model parameters.

import matplotlib.pyplot as plt
import numpy as np
from os import path
import sys

# Ensure project modules are importable (assumes notebook is run from repository root or scripts/ directory)
sys.path.insert(0, path.join('.'))
from modules.leadfield_importer import FieldImporter

# ---------------- User Configuration ---------------- #
# Set the dataset root directory. Must contain:
#   leadfields/  -> with the .npz leadfield file
#   outputs/     -> for saving figures
folder = r"...\SEPIO_dataset"  # TODO: Replace with absolute path to your dataset
fields_file = path.join(folder, 'leadfields', 'DISC_30mm_p2-5Sm_MacChr.npz')
output = path.join(folder, 'outputs')

# ---------------- Load Leadfield ---------------- #
field_importer = FieldImporter()
field = field_importer.load(fields_file)  # Returns loaded field object (if applicable); numeric arrays stored in field_importer.fields
fields = field_importer.fields  # Shape expected: (Nx, Ny, Nz, 3, Nelectrodes)
num_electrodes = fields.shape[4]

# Spatial resolution of the volume (mm per voxel)
scale = 0.5  # mm; voxel size

# ---------------- Dipole & Noise Parameters ---------------- #
# Dipole magnitude (nAm) and noise level (µV RMS). Noise not applied directly here but documented for context.
magnitude = 5e-9  # nAm
noise = 4.1       # µV RMS typical EEG noise reference (not injected in this script)

# After this cell: proceed to calculation of voltages vs. dipole angle.

## Dipole Parameter Notes
The dipole is modeled as a current source with fixed magnitude (`magnitude = 5 nAm`). We rotate its orientation in a vertical plane (x–z) between two DISC devices. Noise level (`noise = 4.1 µV RMS`) is noted for experimental context but not applied in this synthetic sweep.

Location selection:
- Position chosen midway between two devices along +x direction.
- `distance` (defined in next cell) sets half-gap offset in mm.
- The voxel indices are derived from the center of the volume plus this offset.

Orientation sweep:
- Angles vary from 0° (aligned with +x) to 180° (aligned with -x).
- Rotation vector basis: `[cos(a), 0, sin(a)]` for angle `a` in radians.

Electrode selection:
- Three electrodes (indices 0, 2, 4) are sampled to represent distinct montage choices.


In [ ]:
### Dipole Rotation Sweep & Plot ###
# This cell:
# 1. Defines spatial position for dipole between devices.
# 2. Sweeps dipole orientation angle from 0→π radians (0→180°).
# 3. Computes voltage for three electrode montages via dot product of field vector and dipole orientation vector.
# 4. Scales results to microvolts.
# 5. Plots voltage vs. angle with labels and legend.
# 6. Saves figure.
distance = 2.5 #mm; half way between devices


x = int(fields.shape[0]//2 + distance/scale)
y = fields.shape[1]//2
z = fields.shape[2]//2 - 19 # z axis offset
angles = np.linspace(0,np.pi,200) # radians; angles to iterate through
v = np.zeros((3,angles.shape[0]))
for i,a in enumerate(angles): # Calculate the voltage across the given electrodes for each source angle
    v[0,i] = np.dot(fields[x,y,z,:,0],np.array([np.cos(a),0,np.sin(a)]))
    v[1,i] = np.dot(fields[x,y,z,:,2],np.array([np.cos(a),0,np.sin(a)]))
    v[2,i] = np.dot(fields[x,y,z,:,4],np.array([np.cos(a),0,np.sin(a)]))

v *= magnitude*10**6 # Scale to dipole and to uV

angles *= 180/np.pi # Convert to degrees for plotting

plt.plot(angles,v[0],'r')
plt.plot(angles,v[1],'g')
plt.plot(angles,v[2],'b')
plt.plot(angles,np.zeros((angles.shape[0])),'black')
plt.xlim([0,180])

plt.savefig(path.join(output,"4_montage_angle.png"),dpi=300)

## Plot Interpretation
Each colored curve represents voltage measured by a single electrode montage as the dipole rotates from 0° (pointing +x) to 180° (pointing -x).

Key points:
- Peaks occur where dipole orientation aligns most strongly with the local field sensitivity of that electrode.
- Differences between curves illustrate how montage choice alters angular sensitivity.
- The zero line provides a reference; crossings indicate orientations with negligible projected field for that montage.

Next steps / extensions:
- Compare additional electrode indices or bipolar combinations.
- Introduce simulated noise (e.g., add Gaussian with RMS = 4.1 µV) to assess robustness.
- Normalize curves or compute SNR vs. angle.
